# DATA ANALYSIS AND VISUALIZATION FILE
This file is for data analysis and visualization of the data. the data is imported from 'loading_data.ipynb', so make sure the data preprocessing is working correctly within that notebook.

In [1]:
import import_ipynb
import bokeh
from bokeh.models import ColumnDataSource, Select
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.models import CustomJS
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from src.loading_functions import *

In [2]:
#get cwd
cwd = os.getcwd()
#get data folder
data_folder = os.path.join(cwd, 'data')
df = load_or_download_lego_data(data_folder)
df = preprocess_data(df)

Loading data from c:\Users\MGAJ\OneDrive - Danmarks Tekniske Universitet\DTU\Kandidat\4_Semester\Social Data Visualization\MarcusGalea.github.io\data\lego_sets.csv


c:\Users\MGAJ\OneDrive - Danmarks Tekniske Universitet\DTU\Kandidat\4_Semester\Social Data Visualization\MarcusGalea.github.io\src\loading_functions.py:10: DtypeWarning: Columns (34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(os.path.join(data_folder, 'lego_sets.csv'))
c:\Users\MGAJ\OneDrive - Danmarks Tekniske Universitet\DTU\Kandidat\4_Semester\Social Data Visualization\MarcusGalea.github.io\src\loading_functions.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[df.columns[date_columns]] = df[df.columns[date_columns]].applymap(lambda x: x.split("T")[0])


In [3]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
grouped = df.groupby(['Year', 'Month', 'subtheme']).size().reset_index(name='Count')
# Define your threshold
threshold = 7
# Replace X with the desired value
 
# Filter the DataFrame
filtered = grouped[grouped['Count'] > threshold]
#convert dates to strings in filtered
filtered['Year'] = filtered['Year'].astype(str)
filtered['Month']=filtered['Month'].astype(str)
options = filtered['Year'].unique()


C:\Users\MGAJ\AppData\Local\Temp\ipykernel_19856\332638454.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Year'] = filtered['Year'].astype(str)
C:\Users\MGAJ\AppData\Local\Temp\ipykernel_19856\332638454.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month']=filtered['Month'].astype(str)


In [4]:
grouped_allyears = df.groupby(['Date', 'subtheme']).size().reset_index(name='Count')
grouped_allyears = grouped_allyears[grouped_allyears['Count'] > threshold]
#convert datetimeobjects to string
grouped_allyears['Date'] = grouped_allyears['Date'].dt.strftime('%Y-%m-%d')

Loading BokehJS ...

In [6]:
grouped_allyears

,Date,subtheme,Count
1,2018-06-01,3 in 1,33
29,2018-06-01,Buildable Figures,11
40,2018-06-01,DC Comics Super Heroes,8
87,2018-06-01,Licensed,8
96,2018-06-01,Minifig-scale,26
114,2018-06-01,Racing,8
158,2018-06-01,Trains,8
166,2018-07-01,3 in 1,24
172,2018-07-01,Arctic,8
178,2018-07-01,Batman,10


In [14]:
years = df['Year'].unique().astype(str)

In [16]:
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper
from bokeh.plotting import figure, output_file, save
from bokeh.palettes import Viridis256
from math import pi
import pandas as pd

# Create a complete date range for each quarter
start_date = pd.to_datetime(grouped_allyears['Date'].min())
end_date = pd.to_datetime(grouped_allyears['Date'].max())
all_dates = pd.date_range(start_date, end_date, freq='QS').strftime('%Y-%m-%d').tolist()

# Create a figure for each year
subthemes = sorted(grouped_allyears['subtheme'].unique())
for year in years:
    p = figure(x_range=all_dates, y_range=list(reversed(subthemes)), toolbar_location=None, tools="",
               title="All Years Combined")

    p.width = 1200
    p.height = 600

    # Rotate the x-axis labels
    p.xaxis.major_label_orientation = pi/4

    # Create a color mapper
    mapper = LinearColorMapper(palette=Viridis256, low=grouped_allyears.Count.min(), high=grouped_allyears.Count.max())

    # Create a rectangle glyph and add it to the figure
    source = ColumnDataSource(data=grouped_allyears[filtered['Year'] == year])
    p.rect(x="Date", y="subtheme", width=1, height=1, source=source,
           fill_color={'field': 'Count', 'transform': mapper}, line_color=None)

    # Create a color bar
    color_bar = ColorBar(color_mapper=mapper, location=(0, 0), orientation='horizontal')
    p.add_layout(color_bar, 'below')

    # Output to an HTML file
    output_file("year_{}.html".format(year))

    # Save the figure
    save(p)

In [ ]:
filtered

,Year,Month,subtheme,Count
1,2018,6,3 in 1,33
29,2018,6,Buildable Figures,11
40,2018,6,DC Comics Super Heroes,8
87,2018,6,Licensed,8
96,2018,6,Minifig-scale,26
114,2018,6,Racing,8
158,2018,6,Trains,8
166,2018,7,3 in 1,24
172,2018,7,Arctic,8
178,2018,7,Batman,10


In [ ]:
years

['2018', '2019', '2020']

In [38]:
from bokeh.models import Select, ColumnDataSource, ColorBar, LinearColorMapper, CustomJS
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.palettes import Viridis256
from bokeh.io import output_notebook

output_notebook()
output_file("calender_2018.html")

# Create a Select object for the years
years = sorted(filtered['Year'].unique())
select = Select(title="Year", value=str(years[0]), options=[str(year) for year in years])

# Create a figure
subthemes = sorted(filtered['subtheme'].unique())
months = [str(i) for i in range(1, 13)]
p = figure(x_range=months, y_range=list(reversed(subthemes)), toolbar_location=None, tools="", title = "Calender for 2020")

# Create a color mapper
mapper = LinearColorMapper(palette=Viridis256, low=filtered.Count.min(), high=filtered.Count.max())

# Create a rectangle glyph and add it to the figure
source = ColumnDataSource(data=filtered[filtered['Year'] == years[0]])
p.rect(x="Month", y="subtheme", width=1, height=1, source=source,
       fill_color={'field': 'Count', 'transform': mapper}, line_color=None)

# Create a color bar
color_bar = ColorBar(color_mapper=mapper, location=(0, 0), orientation='horizontal')
p.add_layout(color_bar, 'below')


# Add the Select object and the figure to a layout and show it
show(p)

Loading BokehJS ...

In [41]:
from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.palettes import Viridis256
from bokeh.io import output_notebook
from math import pi


output_notebook()
output_file("calendar.html")

# Modify the 'Month' column to include the year
filtered['Month'] = filtered['Year'].astype(str) + ' ' + filtered['Month']

# Create a figure
subthemes = sorted(filtered['subtheme'].unique())
months_years = sorted(filtered['Month'].unique())
p = figure(x_range=months_years, y_range=list(reversed(subthemes)), toolbar_location=None, tools="",
           title="All Years Combined")

# Rotate the x-axis labels
p.xaxis.major_label_orientation = pi/2

# Create a color mapper
mapper = LinearColorMapper(palette=Viridis256, low=filtered.Count.min(), high=filtered.Count.max())

# Create a rectangle glyph and add it to the figure
source = ColumnDataSource(data=filtered)
p.rect(x="Month", y="subtheme", width=1, height=1, source=source,
       fill_color={'field': 'Count', 'transform': mapper}, line_color=None)

# Create a color bar
color_bar = ColorBar(color_mapper=mapper, location=(0, 0), orientation='horizontal')
p.add_layout(color_bar, 'below')

# Show the figure
show(p)

Loading BokehJS ...

C:\Users\MGAJ\AppData\Local\Temp\ipykernel_19856\4157170416.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Month'] = filtered['Year'].astype(str) + ' ' + filtered['Month']
